# CMA-ES


**Disclaimer:** We make use of the implementation available at [PyPi](https://pypi.org/project/cma/) <cite data-cite="pycma"></cite> published by the author Nikolaus Hansen under the BSD license. 


CMA-ES which was proposed in <cite data-cite="cmaes"></cite>. Moreover, a comparing review can be found in <cite data-cite="cmaes-review"></cite>. 
CMA-ES stands for covariance matrix adaptation evolution strategy. Evolution strategies (ES) are stochastic, derivative-free methods for numerical optimization of non-linear or non-convex continuous optimization problems. They belong to the class of evolutionary algorithms and evolutionary computation. An evolutionary algorithm is broadly based on the principle of biological evolution, namely the repeated interplay of variation (via recombination and mutation) and selection: in each generation (iteration) new individuals (candidate solutions) are generated by variation, usually in a stochastic way, of the current parental individuals. Then, some individuals are selected to become the parents in the next generation based on their fitness or objective function value 
$f(x)$. Like this, over the generation sequence, individuals with better and better $f$-values are generated.
(excerpt from [Wikipedia](https://en.wikipedia.org/wiki/CMA-ES)).

### Example

In [1]:
import numpy as np

from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.problems import get_problem
from pymoo.optimize import minimize

problem = get_problem("sphere")

algorithm = CMAES(x0=np.random.random(problem.n_var))

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print(f"Best solution found: \nX = {res.X}\nF = {res.F}\nCV= {res.CV}")


Best solution found: 
X = [0.49999996 0.49999999 0.49999999 0.50000004 0.50000001 0.49999998
 0.50000001 0.50000006 0.50000002 0.49999998]
F = [8.93017523e-15]
CV= [0.]


CMA-ES already has several stopping criteria implemented. However, as for other algorithms, the number of iterations or function evaluations can be directly passed to `minimize`.

In [2]:
res = minimize(problem,
               algorithm,
               ('n_iter', 10),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen  |  n_eval  |     f_avg     |     f_min     |     sigma     | min_std  | max_std  |   axis  
     1 |        1 |  0.9963873547 |  0.9963873547 |  0.1000000000 |  0.10000 |  0.10000 |  1.00005
     2 |       11 |  1.1022132599 |  0.9330233130 |  0.0921977902 |  0.08856 |  0.09333 |  1.18122
     3 |       21 |  1.1193807770 |  0.8609177385 |  0.0893625868 |  0.08460 |  0.09503 |  1.26525
     4 |       31 |  0.9499411639 |  0.8014107861 |  0.0985092987 |  0.09465 |  0.11563 |  1.45222
     5 |       41 |  0.8125152803 |  0.6109576354 |  0.1107246480 |  0.10658 |  0.13672 |  1.65393
     6 |       51 |  0.7700848694 |  0.6072166707 |  0.1229606757 |  0.11680 |  0.16289 |  1.78663
     7 |       61 |  0.6417889253 |  0.4492437379 |  0.1322707219 |  0.12392 |  0.18214 |  1.87742
     8 |       71 |  0.5752648859 |  0.2494472643 |  0.1400117231 |  0.12882 |  0.19407 |  1.95865
     9 |       81 |  0.4874254778 |  0.2263100852 |  0.1413582160 |  0.12895 |  0.19983 |  2.07145
    10 |  

In [3]:
res = minimize(problem,
               algorithm,
               ('n_eval', 50),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen  |  n_eval  |     f_avg     |     f_min     |     sigma     | min_std  | max_std  |   axis  
     1 |        1 |  0.9963873547 |  0.9963873547 |  0.1000000000 |  0.10000 |  0.10000 |  1.00005
     2 |       11 |  1.1022132599 |  0.9330233130 |  0.0921977902 |  0.08856 |  0.09333 |  1.18122
     3 |       21 |  1.1193807770 |  0.8609177385 |  0.0893625868 |  0.08460 |  0.09503 |  1.26525
     4 |       31 |  0.9499411639 |  0.8014107861 |  0.0985092987 |  0.09465 |  0.11563 |  1.45222
     5 |       41 |  0.8125152803 |  0.6109576354 |  0.1107246480 |  0.10658 |  0.13672 |  1.65393
     6 |       51 |  0.7700848694 |  0.6072166707 |  0.1229606757 |  0.11680 |  0.16289 |  1.78663
Best solution found: 
X = [0.72210549 0.21798968 0.50002863 0.4657622  0.89438563 0.45849265
 0.3483714  0.77616195 0.92499636 0.7001058 ]
F = [0.60721667]


Also, easily restarts can be used, which are known to work very well on multi-modal functions. For instance, `Rastrigin` can be solved rather quickly by:

In [4]:
problem = get_problem("rastrigin")

algorithm = CMAES(restarts=10, restart_from_best=True)

res = minimize(problem,
               algorithm,
               ('n_evals', 2500),
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [-1.22569510e-10  2.16619522e-09]
F = [0.]


Our framework internally calls the `cma.fmin2` function. All parameters that can be used there either as a keyword argument or an option can also be passed to the `CMAES` constructor.
An example with a few selected `cma.fmin2` parameters is shown below:

In [5]:
import numpy as np
from pymoo.util.normalization import denormalize

# define an intitial point for the search
np.random.seed(1)
x0 = denormalize(np.random.random(problem.n_var), problem.xl, problem.xu)

algorithm = CMAES(x0=x0,
                 sigma=0.5,
                 restarts=2,
                 maxfevals=np.inf,
                 tolfun=1e-6,
                 tolx=1e-6,
                 restart_from_best=True,
                 bipop=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [2.73861900e-07 3.30861457e-07]
F = [3.65965036e-11]


For more details about hyperparameters, we refer to the software documentation of the `fmin2` in CMA-ES, which can be found [here](http://cma.gforge.inria.fr/apidocs-pycma/cma.evolution_strategy.html#fmin2).
A quick explanation of possible parameters is also provided in the API documentation below.

### API